In [ ]:
import numpy as np
import pandas as pd
import random as rd
import math as m
import csv
from matplotlib import pyplot as plt
import warnings
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture


In [ ]:
# Default parameters
no_of_device = 10
sub_channel = int(no_of_device / 2)
radius = 200
altitude = 100
carrier_frequency = 1 * (10 ** 9)
bandwidth = 1 * (10 ** 6)
alpha = 2 * (10 ** (-28))
mu = (10 ** 3)
eta = 3.76
task_offloading_bits = int(2 * (10 ** 1))
# Task_Offloading_Bits = int(100)
error_propagation = 10 ** (-6)
max_execution_time = 0.01

In [ ]:
# Defining the noise power
def noise_power(bandwidth):
    noise_power_dBm = -174 + 10 * np.log10(bandwidth)
    noise_power = (10 ** (-3)) * (10 ** (noise_power_dBm / 10))
    return noise_power

In [ ]:

# Calculating the transmitting power
def tx_power(tx):
    tx_power = (10 ** (-3)) * (10 ** (tx / 10))
    return tx_power

In [ ]:
# Base Station Location
for i in range(1):
    base_station =[]
    np.random.seed(i)
    arg  = 2 * np.pi * np.random.random(2)
    base_station.append(arg)
base_station = np.array(base_station)
# print(base_station)


In [ ]:
# Device Location w.r.t ground
device_loc =[]
for i in range(no_of_device):
    np.random.seed(i)
    arg = 2 * np.pi * np.random.random(1)
    np.random.seed(i)
    device_r = radius * np.sqrt(np.random.random(1))
    device_x = device_r * np.cos(arg)
    device_y = device_r * np.sin(arg)
    device_loc.append(device_x)
    device_loc.append(device_y)
    device_loc.append(device_r)
device_loc = np.array(device_loc).reshape(no_of_device,3)
# print(device_loc)

# updating device radius w.r.t. the ground base station
for i in range(no_of_device):
    x_axis = device_loc[i][0] - base_station[0][0]
    y_axis = device_loc[i][1] - base_station[0][1]
    distance = np.sqrt(np.square(x_axis) + np.square(y_axis))
    device_loc[i][2] = distance
    # print(x_axis)
    # print(y_axis)
    # print(distance)
# print(device_loc)

# device location w.r.t altitude
for i in range(no_of_device):
    device_loc[i][2] = np.sqrt(np.square(altitude) + np.square(device_loc[i][2]))
# print(device_loc)



In [ ]:
# Creating a dataframe 
columns = ['device_x','device_y','device_r']
dataset = pd.DataFrame(device_loc,columns=columns)
for i in range(no_of_device):
    dataset['base_station_x'] = base_station[0][0]
    dataset['base_station_y'] = base_station[0][1]
dataset = dataset[['base_station_x','base_station_y','device_x','device_y','device_r']]

# Saving the clustering dataframe
# print(dataset.head())
dataset.to_csv('bs_device_dataset.csv',index=False)

In [ ]:
# Fading for each bit transmitted i.e. [no of device * task offloading bits]
channel_gain =[]
abs_channel_gain =[]
for i in range(no_of_device):
    np.random.seed(i)
    channel = np.sqrt(dataset['device_r'][i] ** (-eta)) * (np.random.randn(task_offloading_bits,1) + 1j * np.random.randn(task_offloading_bits,1))
    channel_gain.append(channel)
    abs_channel_gain.append(np.abs(channel)**2)
channel_gain= np.array(channel_gain).reshape(no_of_device,task_offloading_bits)
abs_channel_gain = np.array(abs_channel_gain).reshape(no_of_device,task_offloading_bits)
# channel_gain_df.columns =['channel_gain']
# dataset = pd.concat([dataset, channel_gain_df],axis=1)
# print(channel_gain_df.head())
# print(channel_gain)
# print(np.shape(channel_gain))
# print(channel_gain[0])
# print(channel_gain[0][0])
# print(abs_channel_gain[0])



In [ ]:
# Distributions of UE across the base station
# print(dataset[['device_x','device_y']])
plt.figure(figsize=(8,6))
for i in range(no_of_device):
    plt.scatter(dataset['device_x'][i],dataset['device_y'][i])
plt.scatter(dataset['base_station_x'][0],dataset['base_station_y'][0],s=150,c='black',marker='x',label='BS')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Distribution of UE across Base Station')
plt.legend()
plt.show()
plt.savefig('user_distribution.png')

In [ ]:
# Generating Random Bits for all devices
bit_generation = np.zeros((no_of_device,task_offloading_bits))
for i in range(no_of_device):
    for j in range(task_offloading_bits):
        rd.seed(i+j)
        bit_generation[i][j] = rd.randint(0,1)
# print(bit_generation)
# print(bit_generation[0])
# print(bit_generation[1])
# print(bit_generation[5][0])


In [ ]:
# print(dataset['channel_gain'][1])

In [ ]:
# Modulation  - BPSK
mod_bit =np.zeros((no_of_device,task_offloading_bits))
for i in range(no_of_device):
    for j in range(task_offloading_bits):
        mod_bit[i][j] = (2 * bit_generation[i][j]) - 1
# print(mod_bit)
# print(mod_bit[0])
# print(mod_bit[3][2])

In [ ]:
np.int(no_of_device/2) + 1

In [ ]:
# Find the optimum number of cluster
sse  =[]
silh_score =[]
for k in range(2,(int(no_of_device/2)+ 1)):
    km = KMeans(n_clusters=k, random_state=0,init='k-means++',n_init='auto')
    km.fit(dataset.drop(columns=['channel_gain']))
    label = km.labels_
    sil_score = silhouette_score(dataset.drop(columns=['channel_gain']),label)
    # dataset['Cluster'] = label
    sse.append(km.inertia_)
    silh_score.append(sil_score)
    

In [ ]:
# Forming N/2 or N/2 + 1 Cluster
# k-mean clustering
# sse =[]
km = KMeans(n_clusters=int(no_of_device/2),random_state=0, init='k-means++',n_init='auto')
km.fit(dataset.drop(columns=['channel_gain']))
dataset['kcluster'] =km.labels_

#dbscan clustering
# label with -1 are outliers so will be considered as signel user
dbscan = DBSCAN(eps=50,min_samples=2, algorithm='auto')
dbscan.fit(dataset.drop(columns=['channel_gain']))
dataset['dbcluster'] =dbscan.labels_

#gmm clustering
gmm = GaussianMixture(n_components=int(no_of_device/2),covariance_type='full',init_params ='k-means++',random_state=0)
gmm.fit(dataset.drop(columns=['channel_gain']))
dataset['gmmcluster'] = gmm.predict(dataset.drop(columns=['channel_gain']))



In [ ]:
# K Mean Clustering Plot
km = KMeans(n_clusters=20,random_state=0, init='k-means++',n_init='auto')
km.fit(dataset[['device_x','device_y']])
centroids = km.cluster_centers_
plt.figure(figsize=(8,6))
for i in range(no_of_device):
    plt.scatter(dataset['device_x'][i],dataset['device_y'][i])
plt.scatter(centroids[:,0],centroids[:,1],s=200,c='black',marker='x',label='Centroids')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('K-Mean Clustering')
plt.legend()
plt.show()

In [ ]:
print(sse)

In [ ]:
# Elbow Method and Silhoutte Score Plot
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(range(2,(int(no_of_device/2)+ 1)),sse)
plt.xticks(np.arange(2,(int(no_of_device/2)+ 1),50))
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Cluster')
plt.savefig('Elbow.png')

plt.subplot(1,2,2)
plt.plot(range(2,(int(no_of_device/2)+ 1)),silh_score)
plt.xticks(np.arange(2,(int(no_of_device/2)+ 1),50))
plt.xlabel('Number of Clusters')
plt.ylabel('silhouette_score')
plt.title('silhouette Score for Cluster')
plt.savefig('silhouette_score.png')
plt.show()

In [ ]:
# NOMA Power Allocation

def NOMA_Power(subchannel, index,location):
    noma_power = np.zeros((subchannel, 2), dtype=float)
    for k in range(subchannel):
            if (location[index[k][0]][2] > location[index[k][1]][2]):
                noma_power[k][0] = 0.75
                noma_power[k][1] = 0.25
            else:
                noma_power[k][1] = 0.75
                noma_power[k][0] = 0.25

    return noma_power

In [ ]:
def pol2cart(theta, rho):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

In [ ]:
# UE Locations
lambdaB = 10 ** (-7)
areaTotal = np.pi * (10000)**2
numbPoints = 1 + np.random.poisson(areaTotal * lambdaB)
theta = 2 * np.pi * np.random.rand(numbPoints)
rho = 1 + (10000 - 1) * np.sqrt(np.random.rand(numbPoints))
xx, yy = pol2cart(theta, rho)
print(xx,yy)
points=1
lambdaB1 = np.logspace(-7, -5, points)
print(lambdaB1)


In [ ]:
areaTotal